## Zenodo Metadata Extraction for Digital Commons Batch Upload

### What does this code do?

This code searches Zenodo using the Zenodo API for datasets. There are two available search modes: searching by author affiliation or searching by Zenodo ID (number in the Zenodo URL). It then downloads those metadata records and converts them into a format that can be used for a Digital Commons bulk upload, with appropriate headers. This file will be saved as an .xlsx file and will require manual re-saving to a .xls file for the actual upload. 

### Additional Manual Curation

This file will also require manual curation before it is ready for upload, with special attention to:

- Cleaning up affiliations, ex:
  - Unifying names for a single institution: University of Alabama - Birmingham, UNIVERSITY OF ALABAMA AT BIRMINGHAM
   &rarr; University of Alabama at Birmingham
  - Un-abbreviating institution names: NYU &rarr; New York University 
  - Removing extraneous location details: California Digital Library, Oakland, United States of America &rarr; California Digital Library
- Checking for names and keywords written in all caps
- Checking for special characters or accents that are not formatted properly

### What datasets are included?

We want to include find datasets where at least one author is affiliated with the University if Alabama at Birmingham. Since Zenodo does not use or require ROR IDs, the best way to find these datasets is by searching in the creator affiliation field for the query string "university AND alabama AND birmingham". Since [Zenodo hosts copies of Dryad datasets](https://blog.zenodo.org/2020/03/10/dryad-and-zenodo-our-path-ahead/), this search will locate datasets in both Zenodo and Dryad. As of 11/14/2024, there are 128 total datasets, 41 of which are from Dryad. The Dryad datasets belong to the Zenodo community "dryad", allowing us to isolate them if needed.

### Import the data as a json

The code below uses an access token (you will need to input your own or save it in a text file) and a search query ` "creators.affiliation:(+university +alabama +birmingham)" `. The `size` parameter allows you to choose the number of results returned. We have 128 results, so it is set at 200 to allow for some headroom if new datasets appear.

We are left with `records`, the API response in json format.

In [1]:
import requests


with open('zenodo_access_token.txt', 'r') as file: #save your access token in a txt file
    ACCESS_TOKEN = file.read().rstrip()

# Define search queries for affiliation search and ID-specific search
search_query_affil = 'creators.affiliation:(+university +alabama +birmingham)'
search_query_id = '14629136 + 14602463' # (include more as needed)

# Select which query type you want to use, 'affiliation' or 'id'
query_type_input = 'affiliation' # must be either 'affiliation' or 'id'

def get_records(query_type):  
        if query_type == 'affiliation':
                search_query = search_query_affil
        elif query_type == 'id':
                search_query = search_query_id 
        response = requests.get('https://zenodo.org/api/records/',
                        params={'q': search_query,
                                'access_token': ACCESS_TOKEN,
                                'size': 200, # should be ~128, add headroom
                                'type' : 'dataset'
                                })
        records = response.json()
        return records, query_type

records, query_type = get_records(query_type_input)

print(records)

FileNotFoundError: [Errno 2] No such file or directory: 'zenodo_access_token.txt'

### Generate the necessary dataframe

The data from the API request comes in json format. We want to convert it to a pandas dataframe so that we can work with it more easily. This is done using the [`pd.json_normalize()`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html) function. This results in a dataframe with a *lot* of columns. Some of these columns are irrelevant to our purposes (ex: view and download statistics) so we then remove them using the column names and the `drop()` function. 

#### Date Range for Affiliation Search

You can input a start date by changing the `start_date` variable. This will remove the entries with a publication date earlier than your desired start date, so that you can avoid re-processing entries which you have already entered in your dataset catalog, and potentially manually curated.

The end date (most recent record) defaults to the current date, but you can also set this manually using the `end_date` variable.

These dates refer to the **creation date** of the dataset in the repository. You can change this (ex: if you want to look at the date when a dataset was most recently updated) by inputting a different column header.

Note that this date range will only be affected if the query type is 'affiliation'.

#### Saving

We then save this dataframe in csv format. This allows us to easily view the metadata downloaded from Zenodo. We use the `utf-8-sig` encoding to best preserve symbols and special characters. The name format will be the following:

For affiliation searches: zenodo_expanded_raw_YYYYMMDD_to_YYYYMMDD.csv, where the dates are the start and end dates set by you.
For ID searches: zenodo_expanded_raw_ids_YYYYMMDD.csv, where the date is today's date.


In [15]:
import pandas as pd
import datetime
import os

os.makedirs('raw-data', exist_ok=True)
os.makedirs('batch-upload', exist_ok=True)

def records_to_dataframe(records, unwanted_cols, query_type):
    ### MAKE THE DATAFRAME

    df = pd.json_normalize(records['hits']['hits'][0])

    for i in range(1, len(records['hits']['hits'])):
        df_row = pd.json_normalize(records['hits']['hits'][i])

        df = pd.concat([df, df_row])

    ### DROP UNWANTED COLUMNS

    df = df.drop(columns=unwanted_cols)

    ### Remove records from before a certain date if this is an affiliation search

    if query_type == 'affiliation':
        df = df[df['created'] >= start_date]
        df = df[df['created'] <= end_date]

    ### Save to csv file, named according to query type

    if query_type == 'affiliation':
        date_range = start_date.replace('-', '') + '_to_' + end_date.replace('-', '')
        csv_path = 'raw-data/zenodo_expanded_raw_' + date_range + '.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    elif query_type == 'id':
        csv_path = 'raw-data/zenodo_expanded_raw_ids_' + today.replace('-', '') + '.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    return df

unwanted_cols = ['conceptrecid', 'recid', 'revision', 'files', 'owners', 'status', 'state', 'submitted', 'metadata.title', 'metadata.resource_type.title', 
                 'metadata.resource_type.type', 'metadata.relations.version', 'links.self', 'links.doi', 'links.self_doi', 'links.self_doi_html', 
                 'links.parent', 'links.self_iiif_manifest', 'links.self_iiif_sequence', 'links.files', 'links.media_files', 'links.archive', 'links.archive_media', 
                 'links.latest', 'links.latest_html', 'links.versions', 'links.draft', 'links.reserve_doi', 'links.access_links', 'links.access_grants', 'links.access_users',
                 'links.access_request', 'links.access', 'links.communities-suggestions', 'links.requests', 'stats.downloads', 'stats.unique_downloads',
                 'stats.views', 'stats.unique_views', 'stats.version_downloads', 'stats.version_unique_downloads', 'stats.version_unique_views', 'stats.version_views'
                ]

today = str(datetime.date.today())
start_date = '2025-04-25' # Set your start date here (YYYY-MM-DD)
end_date = today # Change if you wish, 'YYYY-MM-DD' format

df_input = records_to_dataframe(records, unwanted_cols, query_type)

### Helper functions

We will need a series of functions to extract and reformat the information from the dataframe `df_input` and put it into new columns in the dataframe `df_output`, which will then be used to make the Digital Commons batch upload file. 

In [22]:
import csv

def csv_to_dict(file_path):
    '''Imports data from a 2-column csv file, where the first column contains dictionary keys and the second column contains the corresponding values.
    Outputs the resulting dictionary. Used with the relation_types.csv file to generate the strings used for the relation type of a related item.'''
    result_dict = {}
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            key = row[0]  # First column as key
            value = row[1]  # Second column as value
            result_dict[key] = value
    return result_dict

### Creating the relation type dictionary
relation_dict = csv_to_dict('relation_types.csv')

def add_col(df1, col1name, df2, col2name):
    '''Copies a column (col1name) out of df1 and adds it to df2. The name of the column in df2 can be specified using the col2name variable.
    The contents of the column are not altered. Returns df2 with the new column added.'''
    extracted_col = df1[col1name]
    df2 = pd.concat([df2, extracted_col.rename(col2name)], axis=1)
    return df2

def list_to_string(lst):
    '''Takes in a list of strings ['a', 'b', 'c'] and returns a single string containing the list elements, separated by commas 'a, b, c'. '''
    if isinstance(lst, list):  # Check if the value is a list
        return ', '.join(lst)
    else:
        return ""  # Convert non-list values to string or handle as needed

def url_to_html(url, link_text=None):
    '''Converts a url in string form to a html formatted string for a hyperlinked url, with an optional alternate link text.'''
    if not url:
        return ''  # Return empty string if no URL is provided
    link_text = link_text or url  # Use the URL as the link text if no text is provided
    return f'<a href="{url}">{link_text}</a>'

### Dictionary containing the Zenodo terms for licenses as keys, with the values being a list containing the license URL as well as the html formatted text corresponding to each license.
license_dict = {"mit-license" : ["https://opensource.org/license/mit", "<p>This data is available under the MIT License</p>"],
                "cc-zero" : ["https://creativecommons.org/public-domain/cc0/", "<p>This data is public domain under the CC-0.0 License</p>"],
                "cc-by-4.0" : ["http://creativecommons.org/licenses/by/4.0/", "<p>This data is available under the CC-BY 4.0 License</p>"],
                "cc-by-nc-4.0" : ["http://creativecommons.org/licenses/by-nc/4.0/", "<p>This data is available under the CC-BY-NC 4.0 License</p>"],
                "cc-by-2.0" : ["http://creativecommons.org/licenses/by/2.0/", "<p>This data is available under the CC-BY 2.0 License</p>"],
                "cc-by" : ["https://creativecommons.org/licenses/by/1.0/", "<p>This data is available under the CC-BY License</p>"]
}

def add_license(df1, df2):
    '''Adds licensing information columns to df2. Finds the value of the column metadata.license.id in df1
    and uses it as a key for license_dict to retrieve the url fo the license and the string we want displayed in html.
    If there is no value in that row, there is no license shown. We assume this means the data is restricted.'''
    licenses = [] # List for the license url
    access = [] # List for the string/text explaining access
    for license in df1['metadata.license.id']:
        if pd.notnull(license):
            licenses.append(license_dict[license][0])
            access.append(license_dict[license][1])
        else:
            licenses.append('') # Append blank string if there is no license for restricted data
            access.append('<p>Access to this data is restricted.</p>')
    df2['distribution_license'] = licenses
    df2['access_link'] = access
    return df2

def add_repo(df1, df2):
    '''Adds the repository in which the data is stored. Defaults to Zenodo, unless the dataset is in the Dryad community.'''
    repo_list = []
    for community in df1['metadata.communities']: # Iterates through metadata.communities in df1
        if pd.notnull(community) and community[0]['id'] == 'dryad': # If there is a community listed and it is Dryad
            repo_list.append('<p>Dryad</p>')
        else:
            repo_list.append('<p>Zenodo</p>')
    df2['external_rep'] = repo_list
    return df2

def separate_name(name):
    '''Separates a name into First, Middle (if applicable) and Last, returns these elements as separate strings.'''
    middle = ""
    
    # Check if the name contains a comma (indicating "last, first" or "last, first m." format)
    if ',' in name:
        parts = name.split(", ") # Split into a list: "last, first" becomes ["last", "first"]
        last = parts[0] # Last name is everything before the comma
        
        # Check if there's an element after the comma
        if len(parts) > 1:
            first_and_middle = parts[1].split() # Split whatever was after the comma with spaces
            first = first_and_middle[0] # First name will be the first part of that
            
            # Assign middle if available
            if len(first_and_middle) > 1: # If there is a second part
                middle = first_and_middle[1].replace(".", "") # Assign middle name as second part, remove . because DC will add it
        else:
            # Set first to an empty string if there's nothing after the comma
            first = ""
    else: # If the name had no comma, you can assume it is in First Last or First Middle Last format
        parts = name.split() # Split into a list where spaces are 
        
        first = parts[0] # First name is first element of list
        last = parts[-1] # Last name is the last element of the list 
        
        # Check if there is a middle name/initial 
        if len(parts) > 2: 
            middle = parts[1].replace(".", "") # Remove period if it exists
    
    return first, middle, last
    
def reformat_name(name):
    '''Checks if a name string is in "Last, First" format. If it is, returns "First Last".'''
    # Check if the name contains a comma
    if ', ' in name:
        # Split the string by comma and strip any extra whitespace
        last, first = name.split(", ")
        # Return the string in "first last" format
        return f"{first} {last}"
    else:
        # Return the name unchanged if there's no comma
        return name
    
def add_orcid(df1, df2):
    '''Finds authors with ORCIDs and lists them in html format for each dataset, along with hyperlinked urls.'''
    orcid_pairs = [] # List for lists of author/orcid pairs where each element corresponds to a different dataset 
    for index, row in df1.iterrows():
        pairs = [] # List for appending author/orcid pairs within one dataset
        for author in row['metadata.creators']: # Iterate through the list of authors in each row of the metadata.creators column
            if 'orcid' in author:  # Check if ORCID is present
                first_last_name = reformat_name(author['name'])
                pairs.append('<p>' + first_last_name + ' <a href="https://orcid.org/' + author['orcid'] + '">' +author['orcid']+ '</a></p>')
        orcid_pairs.append("".join(pairs))  # Join multiple name-ORCID pairs with a comma
    # Add this list as a new column in df2
    df2['orcid'] = orcid_pairs
    return df2

def to_html(string_input):
    string = str(string_input)
    '''Takes in a string. If the string is not in html already (assume first character is <) wrap it in <p> ... </p>.'''
    if string[0] != "<":
        string = "<p>" + string + "</p>"
    return(string)

def add_funders(df1, df2):
    '''Adds funder information from df1 to df2. Adds funder name, and optionally DOI, grant title, and grant number.'''
    funders = []
    for index, row in df1.iterrows():
        funder = []
        # Iterate through the grants if they are present
        if isinstance(row['metadata.grants'], list):
            for grant in row['metadata.grants']:
                if pd.notnull(grant):  # Check if grant is not null
                    funder.append('<p>Funder: ' + grant['funder']['name'])
                    if 'doi' in grant['funder']:
                        funder.append('<br>Funder DOI: <a href="https://doi.org/' + grant['funder']['doi'] + '">' +grant['funder']['doi']+ '</a>')
                    if 'title' in grant:
                        funder.append('<br>' + grant['title'])
                    if 'code' in grant:
                        funder.append('<br>' + grant['code'])
                    funder.append('</p>')
        else:
            # If it's not a list put an empty cell
            if pd.notnull(row['metadata.grants']):
                funder.append('')
        # Append the joined funder information to the list
        funders.append("".join(funder))
    # Add the new 'fundref' column to df2
    df2['fundref'] = funders
    return df2

def add_related_items(df1, df2):
    '''Adds related items from df1 to df2 with nice html formatting. Includes relation type taken from relation_dict dictionary. Formats PID appropriately if it is a DOI or other URL.'''
    items = []
    for index, row in df1.iterrows():
        item = []
        # Iterate through the related items if they are present
        if isinstance(row['metadata.related_identifiers'], list):
            item.append('<p>')
            count =0
            for id in row['metadata.related_identifiers']:
                if pd.notnull(id):  
                    if count > 0:
                        item.append('<br>')
                    count += 1
                    if id['relation']:
                        item.append(relation_dict[id['relation']] + ': ')
                    if id['scheme'] == 'url':
                        url = id['identifier']
                        item.append( '<a href="'+ url + '">' + url + '</a>')
                    if id['scheme'] == 'doi':
                        doi = id['identifier']
                        item.append('<a href="https://doi.org/' + doi + '">' + doi + '</a>')
                    if id['scheme'] != 'url' and id['scheme'] != 'doi':
                        item.append(id['identifier'])
            item.append('</p>')
        else:
            # If it's not a list put an empty cell
            if pd.notnull(row['metadata.related_identifiers']):
                item.append('')
        # Append the joined funder information to the list
        items.append("".join(item))
    # Add the new 'fundref' column to df2
    df2['related_data'] = items
    return df2

def add_creators(df1, df2):
    '''Adds creator information. Note that this adds an arbitrary number of creators but Digital Commons has a number cap for authors so you may need to manually curate after.
    This function will make a new dataframe df3 and append it on to df2.'''
    # Create a list to hold all rows of data for the new DataFrame
    expanded_data = []
    
    # Process each row in df1
    for _, row in df1.iterrows():
        row_data = {}
        creators = row['metadata.creators']
        
        # Populate the row_data dictionary with each author's name and affiliation
        for i, creator in enumerate(creators):
            author_index = i + 1
            name = creator.get('name', "")
            institution = creator.get('affiliation', "")  # Use 'institution' instead of 'affiliation' (DC nomenclature)
            
            # Use the separate_name function to split names
            first_name, middle_name, last_name = separate_name(name)
            
            # Assign names and institution to the row_data dictionary
            row_data[f'author{author_index}_fname'] = first_name
            row_data[f'author{author_index}_mname'] = middle_name
            row_data[f'author{author_index}_lname'] = last_name
            row_data[f'author{author_index}_institution'] = institution  # Change 'affl' to 'institution'
        
        # Append row_data dictionary to expanded_data list
        expanded_data.append(row_data)

    # Convert expanded_data list of dictionaries into a new DataFrame
    df3 = pd.DataFrame(expanded_data)
    
    # Fill missing values with empty strings for any columns where data is missing
    df3 = df3.fillna("")
    
    df2_reset = df2.reset_index(drop=True)
    df3_reset = df3.reset_index(drop=True)
    
    # Concatenate the two DataFrames along the columns
    df_out = pd.concat([df2_reset, df3_reset], axis=1)
    
    return df_out




Build output dataframe 

In [24]:
### BUILD OUTPUT DATAFRAME

# title
df_output = df_input[['title']]

# orcid
df_output = add_orcid(df1=df_input, df2=df_output)

# publication_date
df_output = add_col(df_input, "metadata.publication_date", df_output, "publication_date")

# abstract
df_output = add_col(df_input, "metadata.description", df_output, "abstract")
df_output['abstract'] = df_output['abstract'].apply(to_html)

# keywords
df_output = add_col(df_input, "metadata.keywords", df_output, "keywords")
df_output["keywords"] = df_output["keywords"].apply(list_to_string)

# disciplines
df_output["disciplines"] = "" #make blank column, we will need to fill in the values

#source_publication
df_output["source_publication"] = "" #make blank column, we will need to fill in the values

# related_data
df_output = add_related_items(df1=df_input, df2=df_output)

# source_fulltext_url
df_output = add_col(df_input, "doi_url", df_output, "source_fulltext_url")

# external_rep
df_output = add_repo(df1=df_input, df2=df_output)

# distribution_license and access_link
df_output = add_license(df1=df_input, df2=df_output)

# funder_info
df_output = add_funders(df1=df_input, df2=df_output)

# author info
df_output = add_creators(df1=df_input, df2=df_output)

display(df_output)

df_output.to_excel('batch-upload/zenodo_batch_upload_' + today + '.xlsx', index=False)

C:\Users\Claire\AppData\Local\Temp\ipykernel_27936\1865036149.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['orcid'] = orcid_pairs


,title,orcid,publication_date,abstract,keywords,disciplines,source_publication,related_data,source_fulltext_url,external_rep,...,author72_lname,author72_institution,author73_fname,author73_mname,author73_lname,author73_institution,author74_fname,author74_mname,author74_lname,author74_institution
0,Chromatin Structural Gene Expression Stratifie...,"<p>Manuel Rosa-Garrido <a href=""https://orcid....",2025-09-02,<p>Chromatin structure plays a central role in...,"HMGN3, Cardiac Disease, Chromatin Structure",,,,https://doi.org/10.5281/zenodo.17035888,<p>Zenodo</p>,...,,,,,,,,,,
1,"Data related to ""Tau, amyloid-beta and alpha-s...","<p>Jhodi Webster <a href=""https://orcid.org/00...",2025-08-22,<h3><strong>Manuscript Abstract</strong></h3>\...,,,,,https://doi.org/10.5281/zenodo.16739908,<p>Zenodo</p>,...,,,,,,,,,,
2,ICTV Master Species List 2024 MSL40.v2,"<p>Elliot Lefkowitz <a href=""https://orcid.org...",2025-08-26,<p>The Master Species List (MSL) provides acce...,,,,,https://doi.org/10.5281/zenodo.16955220,<p>Zenodo</p>,...,,,,,,,,,,
3,Accelerating Drug Repurposing for Rett Syndrom...,,2025-04-25,<p>Above are the datasets that were pulled fro...,"Drug Repurposing, INFO603",,,<p>Is supplemented by: PharmAlchemy</p>,https://doi.org/10.5281/zenodo.15278272,<p>Zenodo</p>,...,,,,,,,,,,
4,Single Cell RNASeq of motor cortex in a mouse ...,"<p>Viktor Feketa <a href=""https://orcid.org/00...",2025-06-12,<p>This Zenodo deposit contains a publicly ava...,,,,,https://doi.org/10.5281/zenodo.15649798,<p>Zenodo</p>,...,,,,,,,,,,
5,Striatal dopamine signals errors in prediction...,"<p>Kauê Costa <a href=""https://orcid.org/0000-...",2025-02-03,<p>nan</p>,,,,,https://doi.org/10.5281/zenodo.14792481,<p>Zenodo</p>,...,,,,,,,,,,
6,Disrupted development of sensory systems and t...,"<p>Summer Thyme <a href=""https://orcid.org/000...",2025-04-29,<p>Behavior and imaging data for the ebf3a mut...,,,,,https://doi.org/10.5281/zenodo.15304810,<p>Zenodo</p>,...,,,,,,,,,,
7,Disrupted Diencephalon Development and Neurope...,"<p>Summer Thyme <a href=""https://orcid.org/000...",2025-05-01,"<p>Imaging, RNA-seq, and behavioral data to su...",,,,,https://doi.org/10.5281/zenodo.15318760,<p>Zenodo</p>,...,,,,,,,,,,
8,"Monoicy, dioicy, and genetic structure in thre...","<p>Sarah Shainker-Connelly <a href=""https://or...",2025-05-09,<p>All multilocus haploid genotypes are provid...,,,,,https://doi.org/10.5281/zenodo.15381199,<p>Zenodo</p>,...,,,,,,,,,,
9,A CLONAL LEGACY? REPRODUCTIVE MODE VARIATION I...,"<p>Alexis Oetterer <a href=""https://orcid.org/...",2025-06-25,<p><span>The </span>red macroalga <em><span>Gr...,,,,,https://doi.org/10.5281/zenodo.15738886,<p>Zenodo</p>,...,,,,,,,,,,
